In [25]:
from ppadb.client import Client as AdbClient
from parsing_usagestats import *
import xml.etree.ElementTree as ET
import pandas as pd
import os
import json

mappings=mappings_parsing()
destination_path = './extractdata//usagestats/daily'
daily_list=os.listdir(destination_path)


Packages=pd.DataFrame(columns=['package_token','package_name','last_time_active_ms','total_time_active_ms','app_launch_count','last_time_visible_ms','total_time_visible_ms'])
EventLog=pd.DataFrame(columns=['package_token','class_token','time_ms','type','package_name','class_name'])


for daily in daily_list:
    mappings_EventLog,mappings_Packages=usagestats_parsing(destination_path,str(daily),mappings)
    if Packages.empty:
        Packages=mappings_Packages
    else:
        Packages=pd.concat([Packages,mappings_Packages])
    if EventLog.empty:
        EventLog=mappings_EventLog
    else:
        EventLog=pd.concat([EventLog,mappings_EventLog])

with open('userBehaviour_type.json','r',encoding='utf-8') as file:
    userBehaviour_type=json.load(file)


EventLog['type_string']=[userBehaviour_type.get(str(type)) for type in EventLog['type']]

In [26]:
keywords = ["wipe", "wiping", "shredde", "shredder", "delete", "shred", "eraser", "securewipe", "eng.bite", "wiper", "remover", "ccleaner","zerdava"]
str_list = []

In [27]:
filtered_Packages = Packages[Packages['package_name'].str.contains('|'.join(keywords), case=False)]
filtered_EventLog = EventLog[EventLog['package_name'].str.contains('|'.join(keywords), case=False)]

In [30]:
filtered_EventLog.to_csv('text.csv')

In [10]:
wiping_application=list(set([package_name for package_name in filtered_Packages['package_name']]))

In [11]:
wiping_application

['com.cbinnovations.androideraser',
 'com.projectstar.ishredder.android.standard',
 'com.palmtronix.shreddit.v1',
 'com.vb2labs.android.sdelete',
 'com.zerdava.fileshredder']

In [3]:
from ppadb.client import Client as AdbClient
from parsing_usagestats import *
import xml.etree.ElementTree as ET
import pandas as pd
import os
import json

In [17]:

destination_path = "./usagestats"

 
package_data = []
eventlog_data=[]
daily_path = os.path.join(destination_path,'0', 'daily')
if os.path.exists(daily_path):
    dailyfile_list=os.listdir(daily_path)
    for file_path in dailyfile_list:
        tree = ET.parse(f'{daily_path}/{file_path}')
        root = tree.getroot()

        for package_element in root.findall('.//package'):
            package_name = package_element.get('package', None)

            timeActive = int(package_element.get('timeActive', 0))
            lastTimeActive=int(package_element.get('lastTimeActive','0'))
            if any(keyword.lower() in package_name.lower() for keyword in keywords) and timeActive > 0 :
                lastTimeActive+=int(file_path)
                package_data.append({
                        'package' : package_name,
                        'timeActive' : timeActive,
                        'lastTimeActive':lastTimeActive
                    })
        for eventlog_element in root.findall('.//event'):
            package_name = eventlog_element.get('package', None)
            class_name=eventlog_element.get('class',None)
            time = int(eventlog_element.get('time', 0))
            type = int(eventlog_element.get('type', 0))
            if any(keyword.lower() in package_name.lower() for keyword in keywords) and time > 0 :
                time+=int(file_path)
                eventlog_data.append({
                        'package':package_name,
                        'class':class_name,
                        'time':time,
                        'type':type
                    })
            

In [20]:
Packages=pd.DataFrame(data=package_data)
EventLog=pd.DataFrame(data=eventlog_data)

In [22]:
timecolumn_list=['lastTimeActive','time']
df_list=Packages,EventLog
for time_column,df in zip(timecolumn_list,df_list):
    df[time_column] = pd.to_datetime(df[time_column], unit='ms', utc=True)
    df[time_column+'_local'] = df[time_column].dt.tz_convert('Asia/Seoul')
    df.drop(columns=[time_column],inplace=True)

In [23]:
Packages

,package,timeActive,lastTimeActive_local
0,com.projectstar.ishredder.android.standard,1982053,2023-11-23 14:19:10.246000+09:00
1,com.shredder.fileshredder.securewipe,4430,2023-11-23 12:17:13.080000+09:00
2,com.projectstar.ishredder.android.standard,1510,2023-11-24 10:29:23.939000+09:00
3,com.zerdava.fileshredder,26465,2023-11-25 16:04:04.729000+09:00
4,com.cbinnovations.androideraser,15526,2023-11-25 16:02:49.596000+09:00
5,com.vb2labs.android.sdelete.pro,3428,2023-11-25 16:03:07.874000+09:00
6,com.projectstar.ishredder.android.standard,3549,2023-11-25 16:02:23.291000+09:00
7,com.palmtronix.shreddit.v1,26794,2023-11-25 16:02:04.086000+09:00
8,com.shredder.fileshredder.securewipe,26178,2023-11-25 16:03:35.306000+09:00
9,com.vb2labs.android.sdelete,1525,2023-11-26 00:17:56.894000+09:00


In [24]:
EventLog

,package,class,type,time_local
0,com.vb2labs.android.sdelete,None,11,2023-11-21 18:39:23.982000+09:00
1,com.vb2labs.android.sdelete.pro,None,11,2023-11-21 18:39:24.002000+09:00
2,com.projectstar.ishredder.android.standard,None,11,2023-11-21 18:39:24.002000+09:00
3,com.shredder.fileshredder.securewipe,None,12,2023-11-21 20:29:30.272000+09:00
4,com.projectstar.ishredder.android.standard,None,11,2023-11-22 05:44:48.617000+09:00
...,...,...,...,...
323,com.shredder.fileshredder.securewipe,None,11,2023-11-28 16:45:18.233000+09:00
324,com.projectstar.ishredder.android.standard,None,11,2023-11-28 17:56:54.688000+09:00
325,com.projectstar.ishredder.android.standard,None,11,2023-11-29 10:02:36.707000+09:00
326,com.projectstar.ishredder.android.standard,None,11,2023-11-29 15:54:47.422000+09:00


In [2]:
import contacts
df=contacts.contacts_wiping('./extractdata/com.samsung.android.providers.contacts/databases','contacts2.db')
df

,local_timestamp,sec_trash_caller_package,cutom_value1,cutom_value2
0,2023-11-21 13:24:44,com.cbinnovations.androideraser,010-1111-1111,NumberOne
1,2023-11-21 13:24:44,com.cbinnovations.androideraser,010-3333-3333,numberThree
2,2023-11-21 13:24:45,com.cbinnovations.androideraser,010-2222-2222,numberTwo
3,2023-11-21 13:35:08,com.projectstar.ishredder.android.standard,000000000000,000000000000 000000000000000000000000000000000...
4,2023-11-21 13:35:09,com.projectstar.ishredder.android.standard,000000000000,000000000000 000000000000000000000000000000000...
5,2023-11-27 16:18:07,com.projectstar.ishredder.android.standard,000000000000,000000000000 000000000000000000000000000000000...
